# Curation Estaciones Meteorologicas

Estudiar La cantidad de datos validos disponibles para cada variable en cada estacion y obtener el numero de estaciones con los datos minimos necesarios para el estudio

In [1]:
setwd("~/Repositories/AirQualityCOVID")

In [2]:
source("src/Curation/general.R")

## Funciones para filtrar datos

In [3]:
suppressMessages(library(lubridate))
suppressMessages(library(tidyverse))
suppressMessages(library(plyr))

In [4]:
get.info <- function(st) {
    
    dataframe <- read.csv(paste("data/Curation/AEMET/Values/",
                                st,
                                ".csv", sep="")) %>%
                    data.as.datetime("fecha", "ymd")
    period <- as.numeric(ymd("2020-12-31")-ymd("2013-01-01")) +2
    
    cbind(data.frame(site=st,
                     start_dt=min(dataframe$fecha),
                     end_dt=max(dataframe$fecha),
                     miss_dy=period - nrow(dataframe)
                    ),
          rbind(apply(dataframe, 2, 
                      function(df, period) {
                          (sum(!is.na(df)) / period)
                      }, period))
         )
}

### Ejecucion del Programa

In [5]:
sitesAQ <- list.files("data/Curation/AEMET/Values/", pattern = ".csv")

all.data <- do.call(rbind.fill, lapply(str_remove(sitesAQ, ".csv"), get.info))
all.data <- all.data[, -which("sol" == names(all.data))]

In [6]:
head(all.data)

,site,start_dt,end_dt,miss_dy,fecha,indicativo,nombre,provincia,altitud,tmed,⋯,tmax,horatmax,dir,velmedia,racha,horaracha,presMax,horaPresMax,presMin,horaPresMin
,<fct>,<date>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0149X,2013-01-01,2020-12-31,42,0.9856312,0.9856312,0.9856312,0.9856312,0.9856312,0.9770783,⋯,0.9770783,0.9856312,0.9322614,0.9404721,0.9322614,0.9856312,0.9784468,0.9784468,0.9784468,0.9784468
2,0200E,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.0000000,1.0000000,0.9993158,1.0000000,0.9993158,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000
3,1059X,2013-01-01,2020-12-31,43,0.9852891,0.9852891,0.9852891,0.9852891,0.9852891,0.9476565,⋯,0.9476565,0.9852891,0.9722887,0.9774205,0.9722887,0.9852891,0.9360246,0.9360246,0.9360246,0.9360246
4,1078I,2013-01-01,2020-12-31,507,0.8265481,0.8265481,0.8265481,0.8265481,0.8265481,0.8029422,⋯,0.8029422,0.8265481,0.8073897,0.8114950,0.8073897,0.8265481,0.6914129,0.6914129,0.6914129,0.6914129
5,1082,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,⋯,1.0000000,1.0000000,0.9996579,0.9993158,0.9996579,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000
6,3129,2013-01-01,2020-12-31,0,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000,0.9996579,⋯,0.9996579,1.0000000,0.9989737,1.0000000,0.9989737,1.0000000,1.0000000,1.0000000,1.0000000,1.0000000


# Guardar Datos

```R
write.csv(all.data,
          "data/Curation/AEMET/info_sites_AEMET.csv",
          row.names=F)

```